In [1]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
import gc
gc.collect()

42

In [3]:
# torch.cuda.empty_cache()

In [4]:
class Opt:
    def __init__(self):
        self.n_epochs = 200
        self.batch_size = 1
        self.lr = 0.002
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim  = 100
        self.n_classes = 10
        self.img_size = 512
        self.channels = 3
        self.sample_interval = 400

In [5]:
opt = Opt()

In [6]:
cuda = True if torch.cuda.is_available() else False
#cuda = False

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.latent_dim)

        self.init_size = opt.img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, opt.n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

In [7]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [8]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (conv_blocks): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ad

In [9]:
# os.makedirs("images", exist_ok=True)

In [10]:
p = r"C:\Users\原神\Downloads\girls"+"\\"
name = "girls"

In [11]:
import os, sys
from PIL import Image

size = (512, 512)

for infile in os.listdir(p+"val\\"+name):
        im = Image.open(p+"val\\"+name+"\\"+infile)
        im = im.resize(size)
        im = im.convert("RGB")
        os.remove(p+"val\\"+name+"\\"+infile)
        im.save(p+"val\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

for infile in os.listdir(p+"train\\"+name):
        im = Image.open(p+"train\\"+name+"\\"+infile)
        im = im.resize(size)
        im = im.convert("RGB")
        os.remove(p+"train\\"+name+"\\"+infile)
        im.save(p+"train\\"+name+"\\"+infile.replace("png","jpg"), "JPEG")

In [12]:
dataset = datasets.ImageFolder(root=p,
                               transform = transforms.Compose(
                                   [
                                       transforms.Resize(opt.img_size), 
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
                                       transforms.Normalize([0.5], [0.5])]),
                              )

In [13]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
)

In [14]:
opt.lr = 0.0002
#opt.lr = 0.001
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

#scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer_D, milestones=[30,80], gamma=0.1)
#scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer_G, start_factor=0.2, end_factor=0.02, total_iters=5000)
#scheduler2 = torch.optim.lr_scheduler.LinearLR(optimizer_D, start_factor=0.2, end_factor=0.02, total_iters=5000)

In [15]:
def sample_image(n_row, batches_done):
    gc.collect()
    torch.cuda.empty_cache()
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, r"C:\Users\原神\Downloads\result1"+"\\%d.png" % batches_done, nrow=n_row, normalize=True)

In [17]:
# opt.lr = 0.00002
# #opt.lr = 0.1
# #opt.lr = 0.01
# # Optimizers
# optimizer_G = torch.optim.SGD(generator.parameters(), lr=opt.lr, weight_decay=1e-5)
# optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=opt.lr, weight_decay=1e-5)

# FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
# LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

# #scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer_G, start_factor=1e0, end_factor=1e-6, total_iters=128)
# #scheduler2 = torch.optim.lr_scheduler.LinearLR(optimizer_D, start_factor=1e0, end_factor=1e-6, total_iters=128)

In [18]:
batches_done = 0

In [19]:
batches_done

0

In [20]:
opt.n_epochs = 5000

In [21]:
# optimizer_G.param_groups[0]['lr'] = 1e-4
# optimizer_D.param_groups[0]['lr'] = 1e-4

In [22]:
opt.sample_interval = 400

In [23]:
for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f] [D lr %.2e][%d]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item(),optimizer_D.param_groups[0]['lr'], batches_done)
            ,end=""
        )
        batches_done += 1
        if batches_done % opt.sample_interval == 0:
               # sample_image(n_row=opt.n_classes, batches_done=batches_done)
               sample_image(n_row=3, batches_done=batches_done)
        
        #gc.collect()
        #torch.cuda.empty_cache()
    #scheduler1.step()
    #scheduler2.step()

C:\Users\原神\AppData\Local\Temp\ipykernel_12020\1369393927.py:7: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)
  valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
C:\ai\lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


[Epoch 25/5000] [Batch 14/17] [D loss: 1.237655, acc: 50%] [G loss: 1.621885] [D lr 2.00e-04][439]]

KeyboardInterrupt: 

In [ ]:
scheduler2.get_last_lr()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
sample_image(n_row=3, batches_done=batches_done)

In [ ]:
# sample_image(n_row=opt.n_classes, batches_done=batches_done)

In [ ]:
gp = r"C:\Users\TEST\generator_girls512rgbclass10_1.pth"
dp = r"C:\Users\TEST\discriminator_girls512rgbclass10_1.pth"

In [ ]:
torch.save(generator.state_dict(), gp)
torch.save(discriminator.state_dict(), dp)

In [ ]:
#generate a random image
sample_image(n_row=1, batches_done=-1)

In [ ]:
discriminator.load_state_dict(torch.load(dp, weights_only=True))
discriminator.eval()

In [ ]:
generator.load_state_dict(torch.load(gp, weights_only=True))
generator.eval()